In [1]:
import pandas as pd
from pymongo import MongoClient

import config
import overalls
import weather
import benedict

In [2]:
pd.set_option('display.max_colwidth', None)
pd.options.display.float_format = '{:,.2f}'.format

In [3]:
def _connect_mongo(uri, db):
    """ A util for making a connection to mongo """

    if uri:
        conn = MongoClient(uri)
    else:
        conn = MongoClient(host, port)
    return conn[db]

def read_mongo_to_df(uri, db, collection, query={}, limit=None):
    """ Read from Mongo and Store into DataFrame """

    con = _connect_mongo(uri, db)
    # Make a query to the specific DB and Collection
    if limit:
        cursor = con[collection].find(query)
        cursor = cursor[limit]
        print(f'number of indexes created has been limited to {limit} ..........................')
    else:
        cursor = con[collection].find(query)
    # Expand the cursor and construct the DataFrame
    df = pd.DataFrame.from_dict(cursor, orient='index')
    return df.transpose()

### ORIGINAL ###
# def forecast_fields(row):
#     row['rain_3h'] = row['rain'].get('3h', 0)
#     row['rain_1h'] = row['rain'].get('1h', row['rain_3h'] / 3)
#     del row['rain']
#     return row
### ORIGINAL ###

### COPY OF ORIGINAL ###
def forecast_fields(row):
    row['rain_3h'] = row['rain'].get('3h', row['rain'].get('1h', 0) * 3)
    row['rain_1h'] = row['rain'].get('1h', row['rain_3h'] / 3)
    del row['rain']
    return row
### COPY OF ORIGINAL ###

def read_mongo_a(uri, db, collection, limit=None, squash=False):
    ''' Retrieve data from the Mongo database and transform it to a pandas
    DataFrame; return the DataFrame.

    :param collection: the collection name
    :type collection: string
    :param limit: optional limiter to the number of documents retrieved. '''
    
    database = _connect_mongo(config.uri, db)
    col = {}
    
    cursor = database[collection].find({})
    # Shorten the cursor length if limit is given, otherwise get everything;
    # transform the retrieved data to a pandas.DataFrame and return it.
    dfs = []
    for doc in cursor[:limit]:
        
        ### trying to add the squash option to the DataFrame fields
        if squash:
            doc['weather'] = benedict.benedict(doc['weather']).flatten()
            doc['forecasts'] = [benedict.benedict(cast) for cast in doc['forecasts']]
        ### trying to add the squash option to the DataFrame fields
        
        dfs.append(
            pd.DataFrame.from_dict([{
                'zipcode': doc['zipcode'],
                'instant': doc['instant'],
                'type': 'weather',
                **doc['weather']
            }], orient='columns')  
### don't need the apply if the dict is flattented-- .apply(forecast_fields, axis=1)
        )
        dfs.append(
            pd.DataFrame.from_dict(
                [{
                    'zipcode': doc['zipcode'],
                    'instant': doc['instant'],
                    'type': 'forecast',
                    **forecast
                 } for forecast in doc['forecasts']]
            ).apply(forecast_fields, axis=1)
        )
    if limit:
        print(f'The length of your df has been limited to {limit}.')

    return pd.concat(dfs).set_index(['zipcode', 'instant', 'time_to_instant'], drop=True)
    
    # Let the user know that even though there were no errors the DataFrame
    # was not created.
    return 'From read_mongo(): There were no errors, but your dataframe was \
    not created.'


def errors(casts, obs):
    ''' Make a dict of errors for the forecasts. Any dicts in the arguments
    will be flattened before comparison.
    
    :param casts: a list of dictionaries
    :param obs: a dictionary
    
    * For best results all dicts should have all the same keys and subkeys.
    '''
    
    # Flatten all dicts and compare. Add the comparisons to a list and return.
    casts = [overalls.flatten_dict(cast) for cast in casts]
    obs = overalls.flatten_dict(obs)
    return [overalls.compare_dicts(cast, obs) for cast in casts]

def gen_errs_df(df):
    ''' Create an errors dataframe from the argument.
    
    :param df: Must be a pandas DataFrame.
    '''
    ### is there a way to step through three lists together? ###
    errs_list = []
    errs_dict = {}
    # Create the error dicts list to be added to the errs_dict.
    for (obs, casts) in zip(df['weather'], df['forecasts']): ### this creates a the list of errors from each instant
        errs_list.append(errors(casts, obs))
    for (_id, errs) in zip(df['_id'], errs_list):  ### this creates a dict from the errors list with the index as key
        errs_dict[_id] = errs
    dd = pd.DataFrame.from_dict(errs_dict, orient='index')
    # Replace the errors DataFrame dictionaries with a list of their values 
    for c in dd.columns:
        dd[c] = [list(d.values()) for d in dd[c]]
    return dd


In [26]:
collection = 'legit_inst'
db = 'owmap'
df = read_mongo_to_df(config.uri, db, collection, limit=5)
df.head(1)
# df = read_mongo_a(config.uri, db, collection, limit=1, squash=False)

number of indexes created has been limited to 5 ..........................


_id     instant zipcode  \
0  5ee355f51525079523946282  1592362800   27006   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 

In [27]:
def weatherize(row):
        row['weather'] = weather.Weather(row['zipcode'], 'observation', row['weather']).weather
        for cast in row['forecasts']:
            cast = weather.Weather(row['zipcode'], 'forecast', cast).weather
        return row

In [30]:
df = df.apply(weatherize, axis=1)

## Convert individual instants to dictionaries(or csv)

In [5]:
df.to_csv('test.csv')

### Create the error docs

In [31]:
for w in df['weather']:
    w.pop('sunset_time', 'sunrise_time')
    w.pop('sunrise_time')
for l in df['forecasts']:
    for f in l:
        f.pop('sunset_time', 'sunrise_time')
        f.pop('sunrise_time')


In [32]:
# err_dict = compare_dicts()
errs = []
for index, row in df[['forecasts', 'weather']].iterrows():
     errs.append(errors(row['forecasts'], row['weather']))
df['errs'] = errs
# df.head()

In [33]:
def strip_keys(dict_list):
    ''' Take a list of dictionaries and return a  list of lists of those
    dictionary values. 
    
    :param e: a list of dicts
    :type e: At least in forecast-forecast the input type will be a pandas
    Series, but in general it can take any list-like object of dictionaries.
    :return: a list of lists of dict values w/o keys
    '''
    
    values = []
    for d in dict_list:
        values.append([list(overalls.flatten_dict(dic).values()) for dic in d])
    return values

err_vals = strip_keys(df['errs'])
cast_vals = strip_keys(df['forecasts'])
dd = pd.DataFrame([cast_vals, err_vals], index=['forecasts', 'errors'])
dd = dd.transpose()


In [34]:
dd.head()

forecasts  \
0  [[100, 1, 4.12, 33, 96, 1021, 1021, 288.39, 0, 288.39, 288.39, Rain, light rain, 500, 10n, None, None, None, None, 405836], [100, 4.36, 31, 90, 1022, 1022, 287.92, 0, 287.92, 287.92, Clouds, overcast clouds, 804, 04n, None, None, None, None, 391434], [100, 4.36, 31, 90, 1022, 1022, 287.92, 0, 287.92, 287.92, Clouds, overcast clouds, 804, 04n, None, None, None, None, 387838], [100, 4.36, 31, 90, 1022, 1022, 287.92, 0, 287.92, 287.92, Clouds, overcast clouds, 804, 04n, None, None, None, None, 380637], [100, 4.36, 31, 90, 1022, 1022, 287.92, 0, 287.92, 287.92, Clouds, overcast clouds, 804, 04n, None, None, None, None, 377038], [100, 4.36, 31, 90, 1022, 1022, 287.92, 0, 287.92, 287.92, Clouds, overcast clouds, 804, 04n, None, None, None, None, 373437], [98, 3.61, 45, 95, 1024, 1024, 287.29, 0, 287.29, 287.29, Clouds, overcast clouds, 804, 04n, None, None, None, None, 369838], [98, 3.61, 45, 95, 1024, 1024, 287.29, 0, 287.29, 287.29, Clouds, overcast clouds, 804, 04n, None, None, None, None, 366238], [98, 3.61, 45, 95, 1024, 1024, 287.29, 0, 287.29, 287.29, Clouds, overcast clouds, 804, 04n, None, None, None, None, 362637], [98, 3.61, 45, 95, 1024, 1024, 287.29, 0, 287.29, 287.29, Clouds, overcast clouds, 804, 04n, None, None, None, None, 359037], [98, 3.61, 45, 95, 1024, 1024, 287.29, 0, 287.29, 287.29, Clouds, overcast clouds, 804, 04n, None, None, None, None, 355437], [98, 3.61, 45, 95, 1024, 1024, 287.29, 0, 287.29, 287.29, Clouds, overcast clouds, 804, 04n, None, None, None, None, 351838], [100, 1.61, 3.75, 46, 96, 1023, 1023, 287.79, 0, 287.79, 287.79, Rain, light rain, 500, 10n, None, None, None, None, 348237], [100, 1.61, 3.75, 46, 96, 1023, 1023, 287.79, 0, 287.79, 287.79, Rain, light rain, 500, 10n, None, None, None, None, 344638], [100, 1.61, 3.75, 46, 96, 1023, 1023, 287.79, 0, 287.79, 287.79, Rain, light rain, 500, 10n, None, None, None, None, 341037], [100, 1.61, 3.75, 46, 96, 1023, 1023, 287.79, 0, 287.79, 287.79, Rain, light rain, 500, 10n, None, None, None, None, 337437], [100, 1.61, 3.75, 46, 96, 1023, 1023, 287.79, 0, 287.79, 287.79, Rain, light rain, 500, 10n, None, None, None, None, 333838], [100, 1.61, 3.75, 46, 96, 1023, 1023, 287.79, 0, 287.79, 287.79, Rain, light rain, 500, 10n, None, None, None, None, 330238], [97, 0.16, 3.98, 36, 90, 1024, 1024, 287.93, 0, 287.93, 287.93, Rain, light rain, 500, 10n, None, None, None, None, 326637], [97, 0.16, 3.98, 36, 90, 1024, 1024, 287.93, 0, 287.93, 287.93, Rain, light rain, 500, 10n, None, None, None, None, 323038], [97, 0.16, 3.98, 36, 90, 1024, 1024, 287.93, 0, 287.93, 287.93, Rain, light rain, 500, 10n, None, None, None, None, 319437], [97, 0.16, 3.98, 36, 90, 1024, 1024, 287.93, 0, 287.93, 287.93, Rain, light rain, 500, 10n, None, None, None, None, 315837], [100, 3.1, 5.05, 39, 96, 1023, 1023, 287.93, 0, 287.93, 287.93, Rain, moderate rain, 501, 10n, None, None, None, None, 308636], [100, 3.1, 5.05, 39, 96, 1023, 1023, 287.93, 0, 287.93, 287.93, Rain, moderate rain, 501, 10n, None, None, None, None, 301437], [100, 3.53, 46, 94, 1024, 1024, 289.13, 0, 289.13, 289.13, Clouds, overcast clouds, 804, 04n, None, None, None, None, 236638], [100, 3.53, 46, 94, 1024, 1024, 289.13, 0, 289.13, 289.13, Clouds, overcast clouds, 804, 04n, None, None, None, None, 233996], [100, 3.53, 46, 94, 1024, 1024, 289.13, 0, 289.13, 289.13, Clouds, overcast clouds, 804, 04n, None, None, None, None, 223197], [100, 0.92, 3.74, 23, 95, 1024, 1024, 287.72, 0, 287.72, 287.72, Rain, light rain, 500, 10n, None, None, None, None, 212393], [100, 0.92, 3.74, 23, 95, 1024, 1024, 287.72, 0, 287.72, 287.72, Rain, light rain, 500, 10n, None, None, None, None, 201594], [100, 0.12, 4.3, 43, 95, 1024, 1024, 288.8, 0, 288.8, 288.8, Rain, light rain, 500, 10n, None, None, None, None, 190794], [100, 5.59, 4.8, 36, 96, 1023, 1023, 288.34, 0, 288.34, 288.34, Rain, moderate rain, 501, 10n, None, None, None, None, 169193], [100, 5.59, 4.8, 36, 96, 1023, 1023, 288.34, 0, 288.34, 288.34, Rain, mode

In [35]:
filename = 'error_set.csv'
dd.to_csv(filename, float_format='%.2f')